In [1]:
import numpy as np
import pandas as pd
import os

os.getcwd()

'C:\\Users\\filip\\Desktop\\Data science\\Projekty\\Gw2 Flipping Optimizer'

In [2]:
## Reading data
buy_data_path = 'data/buy_orders/buy_history_2025-12-15.csv'
sell_data_path = 'data/sell_orders/sell_history_2025-12-15.csv'

## Buys

In [4]:
buys = pd.read_csv(buy_data_path)
buys.head()


,item_id,item_name,quantity,price_copper,created,purchased,time_to_fill_hours
0,27328,Ebonblade,1,4347,2025-12-14T19:48:47+00:00,2025-12-15T11:06:04+00:00,15.29
1,24575,Superior Sigil of Bloodlust,1,1278,2025-12-14T15:24:09+00:00,2025-12-15T11:05:16+00:00,19.69
2,25990,Serpentstone,1,2298,2025-12-13T12:25:03+00:00,2025-12-15T11:00:56+00:00,46.60
3,27485,Deathwish,1,2345,2025-12-14T15:26:26+00:00,2025-12-15T10:58:42+00:00,19.54
4,25990,Serpentstone,1,2298,2025-12-13T12:25:03+00:00,2025-12-15T10:44:09+00:00,46.32


In [5]:
buys.groupby('item_name').agg({'quantity':'sum'}).sort_values('quantity', ascending = False)

,quantity
item_name,
Superior Sigil of Bloodlust,12
Unity,7
Mist-Infused Master Tuning Crystal,7
Deathwish,6
Saltspray Axe,4
Usoku's Needle,4
Grieving Consecrated Saryx Dagger,4
Impaler,4
Serpentstone,4


## Sells

In [7]:
sells = pd.read_csv(sell_data_path)
sells.head()

,item_id,item_name,quantity,price_copper,created,purchased,time_to_fill_hours
0,26152,Usoku's Needle,1,9992,2025-12-12T10:10:08+00:00,2025-12-15T11:13:26+00:00,73.06
1,96760,Spindrift Hammer,1,3853,2025-12-14T15:04:54+00:00,2025-12-15T10:57:51+00:00,19.88
2,96542,Spindrift Dagger,1,4263,2025-12-14T15:05:14+00:00,2025-12-15T09:52:29+00:00,18.79
3,96542,Spindrift Dagger,1,4263,2025-12-14T15:05:14+00:00,2025-12-15T09:45:46+00:00,18.68
4,70772,Recipe: Svaard's Masque,1,3190,2025-12-14T19:36:43+00:00,2025-12-15T09:42:48+00:00,14.10


In [8]:
sells.groupby('item_name').agg({'quantity':'sum'}).sort_values('quantity', ascending = False)

,quantity
item_name,
Toy Sword Skin,500
Serpentstone,6
Norgu's Coat,6
Adam,5
Deathwish,5
Crystalline Blade,4
Spindrift Dagger,4
Demure Dye,3
Spindrift Hammer,3


In [9]:
import glob
# Combines all buy history files
all_files = glob.glob('data/buy_orders/buy_history_*.csv')
dfBuy = pd.concat([pd.read_csv(f) for f in all_files])
dfBuy = dfBuy.drop_duplicates()  # remove any duplicates

# Combines all sell history files
all_files = glob.glob('data/sell_orders/sell_history_*.csv')
dfSell = pd.concat([pd.read_csv(f) for f in all_files])
dfSell = dfSell.drop_duplicates()  # remove any duplicates

In [10]:
bought_demo = ['100 of sigil of bloodlust 8. 12.2025', '24 of sigil of bloodlust 8. 12.2025']
sold_demo = ['50 of sigil of bloodlust 8. 12.2025', '23 of sigil of bloodlust 9. 12.2025', '11 of sigil of bloodlust 10. 12.2025', '2 of sigil of bloodlust 12. 12.2025', '33 of sigil of bloodlust 16. 12.2025' ]#put the last one on purpose more than 7 days in the future
#not sure if i should include the price it was filled, guess it doesnt hurt but I usually sell at market price -1 copper to squeeze out margin. Like maybe the model could predict a sale faster at a different price point, but eh, that seems more nice to have. What I want is to know how fast its gonna sell at market price-1c, so i guess for this use case it might not be important

#Survival analysis with Weibull Distribution

$$f(t) = \frac{k}{\lambda} \left( \frac{t}{\lambda} \right)^{k-1} e^{-\left( \frac{t}{\lambda} \right)^k}$$

This function shows how failure probability density changes over time, with the shape parameter k determining if failures are more likely early (k < 1), constant over time (k = 1), or increasing with age (k > 1).

The cumulative distribution function (CDF) gives the probability that failure occurs by a specific time:
$$F(t) = 1 - e^{-\left( \frac{t}{\lambda} \right)^k}$$

This function works well for calculating the percentage of items expected to fail within a given timeframe. This is what I need for my use case. I need to see if, say, 90% get sold ('fail') within a week. However, is this the same thing as probability to fill? I feel an itch to use the Bayes theorem but I am not completely sure.

 The reliability function (1-CDF) provides the complementary perspective, showing the probability of survival beyond time t.

The hazard function reveals the instantaneous failure rate at any given time. This is useful to model undercut probability after a certain time period should I ever need this.

$$h(t) = \frac{k}{\lambda} \left( \frac{t}{\lambda} \right)^{k-1}$$

In [13]:
from lifelines.fitters.weibull_fitter import WeibullFitter
#!pip install lifelines
from lifelines import WeibullFitter as wf
import json
from datetime import datetime

def fit_item_models(df,min_observations=3):
  #@ min observations is the minimum number of items sold to fit a model
  """  FIts WEibull distribution on each item in the list given enough data"""
  item_distributions = {}

  # Groupby item
  for item_id, group in df.groupby('item_name'):
      if len(group) < min_observations: continue
      item_name = group['item_name'].iloc[0]
      durations = group['time_to_fill_hours'].dropna()

      #WEibull fit
      wf = WeibullFitter()
      wf.fit(durations)

      item_distributions[int(item_id)] = {
          'item_name':item_name,
          'lambda_': float(wf.lambda_),#scale parameter
          'rho_': float(wf.rho_),#shape parameter
          'n_observations': len(durations),
          'median_fill_hours': float(durations.median()),
          'mean_fill_hours': float(durations.mean()),
          'std_fill_hours': float(durations.std()),
      }
  return item_distributions


In [14]:
#Fit models
print("Fitting models  per item...")
models = fit_item_models(dfSell,min_observations=3)
print("Done for {len(models)} items")
#Save to JSON
with open('data/item_distributions.json', 'w') as f:
    json.dump(models, f, indent=2)
print("Models saved")

Fitting models  per item...


ValueError: invalid literal for int() with base 10: 'Adam'

In [ ]:
def calculate_fill_probability(lambda_, rho_, hours):
    """
    Calculate P(item fills within 'hours' hours) using Weibull CDF
    F(t) = 1 - exp(-(t/lambda)^rho)
    """
    return 1 - np.exp(-((hours / lambda_) ** rho_))

# Example: What's the probability each item fills within 24 hours?
time_horizon_hours = 24

print(f"\nFill probabilities within {time_horizon_hours} hours:")
for item_id, model in list(models.items())[:10]:  # Show first 10
    prob = calculate_fill_probability(model['lambda_'], model['rho_'], time_horizon_hours)
    print(f"{model['item_name']}: {prob:.1%} (shape={model['rho_']:.2f})")

#Optimizer
I guess this should be a separate file later? If I do end up saving the probabilities somewhere to save time and recalculate only occasionally.

In [ ]:
#For optimizer
budget = 1000 #gold
time_horizon = 1 #days
min_margin=.05
min_fill_prob = .9
max_transactions = 50 #batch of up to 250 items. This is to prevent optimizer solutions that would have me click through 1000 buy orders.

z = ...#sum of sell price of items meeting the margin threshold and min_fill_prob
full_item_list = pd.read_csv('data/my_trading_items.csv')[["item_name"]] #not even sure what I did this for yet
print(full_item_list)

